# CNN Model - 2018 Paper (Kachuee, Fazeli, Sarrafzadeh), original

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, MaxPooling1D, Flatten, Add, ReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
)
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE, RandomOverSampler

import pickle
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print(tf.config.list_physical_devices('GPU'))  # should show []
from contextlib import redirect_stdout
import json

2025-10-30 11:26:47.164739: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-30 11:26:47.208161: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-30 11:26:48.605846: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


[]


2025-10-30 11:26:49.388710: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
SAMPLING_METHOD = "SMOTE"
REMOVE_OUTLIERS = False
model_name = "cnn6_sm_lr_bs"
OUTPUT_PATH = "src/models/CNN/"
results_csv = "reports/03_model_testing_results/05_CNN_model_comparison.csv"
EPOCHS = 50

#import MIT data
df_mitbih_test = pd.read_csv('data/original/mitbih_test.csv', header = None)

X_train = pd.read_csv('data/processed/mitbih/X_train.csv')
y_train = pd.read_csv('data/processed/mitbih/y_train.csv')
y_train = y_train['187']

X_train_sm = pd.read_csv('data/processed/mitbih/X_train_sm.csv')
y_train_sm = pd.read_csv('data/processed/mitbih/y_train_sm.csv')
y_train_sm = y_train_sm['187']

X_val = pd.read_csv('data/processed/mitbih/X_val.csv')
y_val = pd.read_csv('data/processed/mitbih/y_val.csv')
y_val = y_val['187']

X_test = df_mitbih_test.drop(187, axis = 1)
y_test = df_mitbih_test[187]


# Reshape the data for 1D CNN
X_train_sm_cnn = np.expand_dims(X_train_sm, axis=2)
X_val_cnn = np.expand_dims(X_val, axis=2)
X_test_cnn = np.expand_dims(X_test, axis=2) 

display(X_train_sm_cnn.shape)
display(X_val_cnn.shape)
display(X_test_cnn.shape)

(289885, 187, 1)

(17511, 187, 1)

(21892, 187, 1)

In [3]:
#Function to plot and save validation accuracy and validation loss over epochs from history
def plot_training_history(history, save_dir, prefix): 
    hist = history.history
    metrics = [m for m in hist.keys() if not m.startswith('val_')]  

    # Create the output folder if it does not exist
    os.makedirs(save_dir, exist_ok=True)

    for m in metrics:
        plt.figure()
        plt.plot(hist[m], label=f'Train {m}')
        if f'val_{m}' in hist:
            plt.plot(hist[f'val_{m}'], label=f'Val {m}')
        plt.xlabel('Epoch')
        plt.ylabel(m)
        plt.title(f'{m} over epochs')
        plt.legend()
        plt.grid(True)

        # Construct filename with prefix and filepath with directory and filename
        filename = f"{prefix}_{m}.png"
        filepath = os.path.join(save_dir, filename)

        # Save the figure
        plt.savefig(filepath, format='png', dpi=300, bbox_inches='tight')
        print(f"Saved: {filepath}")
        plt.show()

In [4]:
#CNN6, Paper 2018
# Input layer
input_layer = Input(shape=(187, 1))

conv_0 = Conv1D(filters=32, kernel_size=5, padding='same')(input_layer)

# First Residual Block
conv1_1 = Conv1D(filters=32, kernel_size=5, padding='same')(conv_0)
relu1_1 = ReLU()(conv1_1)
conv2_1 = Conv1D(filters=32, kernel_size=5, padding='same')(relu1_1)
skip_connection_1 = Add()([conv_0, conv2_1])
relu2_1 = ReLU()(skip_connection_1)
pool_1 = MaxPooling1D(pool_size=5, strides=2, padding='same')(relu2_1)

# Second Residual Block
conv1_2 = Conv1D(filters=32, kernel_size=5, padding='same')(pool_1)
relu1_2 = ReLU()(conv1_2)
conv2_2 = Conv1D(filters=32, kernel_size=5, padding='same')(relu1_2)
skip_connection_2 = Add()([pool_1, conv2_2])
relu2_2 = ReLU()(skip_connection_2)
pool_2 = MaxPooling1D(pool_size=5, strides=2, padding='same')(relu2_2)

# Third Residual Block
conv1_3 = Conv1D(filters=32, kernel_size=5, padding='same')(pool_2)
relu1_3 = ReLU()(conv1_3)
conv2_3 = Conv1D(filters=32, kernel_size=5, padding='same')(relu1_3)
skip_connection_3 = Add()([pool_2, conv2_3])
relu2_3 = ReLU()(skip_connection_3)
pool_3 = MaxPooling1D(pool_size=5, strides=2, padding='same')(relu2_3)

# Fourth Residual Block
conv1_4 = Conv1D(filters=32, kernel_size=5, padding='same')(pool_3)
relu1_4 = ReLU()(conv1_4)
conv2_4 = Conv1D(filters=32, kernel_size=5, padding='same')(relu1_4)
skip_connection_4 = Add()([pool_3, conv2_4])
relu2_4 = ReLU()(skip_connection_4)
pool_4 = MaxPooling1D(pool_size=5, strides=2, padding='same')(relu2_4)

# Fifth Residual Block
conv1_5 = Conv1D(filters=32, kernel_size=5, padding='same')(pool_4)
relu1_5 = ReLU()(conv1_5)
conv2_5 = Conv1D(filters=32, kernel_size=5, padding='same')(relu1_5)
skip_connection_5 = Add()([pool_4, conv2_5])
relu2_5 = ReLU()(skip_connection_5)
pool_5 = MaxPooling1D(pool_size=5, strides=2, padding='same')(relu2_5)

# Fully connected layers
flatten = Flatten()(pool_5)
fc1 = Dense(32, activation='relu')(flatten)
output_layer = Dense(5, activation='softmax')(fc1)


# Model
cnn6 = Model(inputs=input_layer, outputs=output_layer)

In [5]:
cnn6.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 187, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 187, 32)   │        192 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 187, 32)   │      5,152 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 187, 32)   │          0 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 187, 32)   │      5,152 │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 187, 32)   │          0 │ conv1d[0][0],     │
│                     │                   │            │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 187, 32)   │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 94, 32)    │          0 │ re_lu_1[0][0]     │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 94, 32)    │      5,152 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 94, 32)    │          0 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 94, 32)    │      5,152 │ re_lu_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 94, 32)    │          0 │ max_pooling1d[0]… │
│                     │                   │            │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 94, 32)    │          0 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 47, 32)    │          0 │ re_lu_3[0][0]     │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 47, 32)    │      5,152 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 47, 32)    │          0 │ conv1d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 47, 32)    │      5,152 │ re_lu_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 47, 32)    │          0 │ max_pooling1d_1[… │
│                     │                   │            │ conv1d_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_5 (ReLU)      │ (None, 47, 32)    │          0 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_2     │ (None, 24, 32)    │          0 │ re_lu_5[0][0]     │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 24, 32)    │      5,152 │ max_pooling1d_2[… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 58,053 (226.77 KB)

 Trainable params: 58,053 (226.77 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Learning rate with exponential decay
initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.75,
    staircase=True
)

# Adam optimizer with specified hyperparameters
optimizer = Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999
)

# Compile the model
cnn6.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)


# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath=OUTPUT_PATH+model_name+'_bs_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5)
    monitor='val_loss',        # metric to monitor
    mode='min',                    # because higher accuracy is better
    save_best_only=True,           # only save when val_accuracy improves
    verbose=1                      # print message when a model is saved
)

In [ ]:
history = cnn6.fit(
    X_train_sm_cnn,
    y_train_sm,
    epochs=EPOCHS,
    batch_size=128,
    validation_data=(X_val_cnn, y_val),  
    callbacks=[checkpoint]
)

Epoch 1/50
2263/2265 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8834 - loss: 0.3230
Epoch 1: val_loss improved from None to 0.23280, saving model to src/models/CNN/cnn6_sm_lr_bs_bs_epoch_01_valloss_0.2328.keras
2265/2265 ━━━━━━━━━━━━━━━━━━━━ 49s 21ms/step - accuracy: 0.9358 - loss: 0.1815 - val_accuracy: 0.9195 - val_loss: 0.2328
Epoch 2/50
  70/2265 ━━━━━━━━━━━━━━━━━━━━ 46s 21ms/step - accuracy: 0.9725 - loss: 0.0823

In [ ]:
cnn6.save(OUTPUT_PATH + model_name + '.keras')

In [ ]:
#Save training history
import json 
SAMPLING_METHOD = "SMOTE"
REMOVE_OUTLIERS = False
model_name = "cnn6_sm_lr_bs"
OUTPUT_PATH = "src/models/CNN/"
results_csv = "reports/03_model_testing_results/05_CNN_model_comparison.csv"


In [ ]:
from pathlib import Path
import re 
def parse_epoch_from_name(name, default_epochs=EPOCHS):
    # Expect pattern like ..._epoch_12_...; returns int if found else default
    m = re.search(r"epoch_(\d+)", name)
    return int(m.group(1)) if m else default_epochs

# Safer file filtering
model_dir = Path(OUTPUT_PATH)
model_paths = sorted([p for p in model_dir.glob("*.keras")])

all_labels = np.unique(y_test)  # ground-truth labels present in test set
rows = []

print(all_labels)

for p in model_paths:
    print(p)
    model_ = load_model(str(p))

    y_pred = model_.predict(X_test_cnn)
    y_pred_class = np.argmax(y_pred, axis=1)

    # Force consistent label space for metrics
    print(classification_report(y_test, y_pred_class, digits=4))
    report = classification_report(
        y_test, y_pred_class, labels=all_labels, output_dict=True, zero_division=0
    )

    print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))

    accuracy = accuracy_score(y_test, y_pred_class)
    epoch_num = parse_epoch_from_name(p.name)

    row = {
        "sampling_method": SAMPLING_METHOD,
        "outliers_removed": REMOVE_OUTLIERS,
        "epochs": epoch_num,
        "model": p.name,
        "test_accuracy": round(float(accuracy), 4),
        "test_f1_macro": round(float(report["macro avg"]["f1-score"]), 4),
    }
    for lbl in all_labels:
        row[f"test_f1_cls_{int(lbl)}"] = round(float(report[str(lbl)]["f1-score"]), 4)

    rows.append(row)

df = pd.DataFrame(rows)
os.makedirs(os.path.dirname(results_csv), exist_ok=True)
df.to_csv(results_csv, index=False)
"""
for model_name in model_paths:
    epoch = model_name.find("epoch_")
    if epoch == -1:
        epoch = EPOCHS
    elif epoch >=0:
        pos = epoch+len("epoch_")
        epoch = model_name[pos:pos+model_name.find("_")-2]

    model_ = load_model(OUTPUT_PATH + model_name) #change for model
    

    #prediction of test data
    y_pred = model_.predict(X_test_cnn)
    y_test_class = y_test
    y_pred_class = np.argmax(y_pred, axis=1)

    print(classification_report(y_test_class, y_pred_class, digits=4))
    report = classification_report(y_test_class, y_pred_class, output_dict=True, zero_division=0)

    #confusion matrix
    print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))

    labels = np.unique(np.concatenate([y_train, y_val]))

    accuracy = accuracy_score(y_test_class, y_pred_class)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_test_class, y_pred_class, average='macro', zero_division=0
    )

    # Ensure consistent label ordering
    labels = np.unique(np.concatenate([y_train, y_val]))

    row = {
        'sampling_method': SAMPLING_METHOD,
        'outliers_removed': REMOVE_OUTLIERS,
        'epochs': epoch,
        'model': model_name,
        'test_accuracy': round(float(accuracy), 4),
        'test_f1_macro': round(float(report['macro avg']['f1-score']), 4),
    }

    # Per-class F1s
    for lbl in labels:
        row[f'test_f1_cls_{lbl}'] = round(float(report[str(lbl)]['f1-score']), 2)

    os.makedirs(os.path.dirname(results_csv), exist_ok=True)
    header = not os.path.exists(results_csv)
    pd.DataFrame([row]).to_csv(results_csv, mode='a', index=False, header=header)"""

[0. 1. 2. 3. 4.]
src/models/CNN/cnn6_sm_lr_bs.keras
685/685 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
              precision    recall  f1-score   support

         0.0     0.9922    0.9946    0.9934     18118
         1.0     0.8731    0.8417    0.8571       556
         2.0     0.9666    0.9599    0.9633      1448
         3.0     0.8366    0.7901    0.8127       162
         4.0     0.9931    0.9907    0.9919      1608

    accuracy                         0.9866     21892
   macro avg     0.9323    0.9154    0.9237     21892
weighted avg     0.9864    0.9866    0.9865     21892

Predictions      0    1     2    3     4
187                                     
0.0          18020   60    23    8     7
1.0             77  468    10    0     1
2.0             33    5  1390   17     3
3.0             19    1    14  128     0
4.0             12    2     1    0  1593
src/models/CNN/cnn6_sm_lr_bs_epoch_01_valloss_0.1284.keras
685/685 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
              precision    reca

'\nfor model_name in model_paths:\n    epoch = model_name.find("epoch_")\n    if epoch == -1:\n        epoch = EPOCHS\n    elif epoch >=0:\n        pos = epoch+len("epoch_")\n        epoch = model_name[pos:pos+model_name.find("_")-2]\n\n    model_ = load_model(OUTPUT_PATH + model_name) #change for model\n    \n\n    #prediction of test data\n    y_pred = model_.predict(X_test_cnn)\n    y_test_class = y_test\n    y_pred_class = np.argmax(y_pred, axis=1)\n\n    print(classification_report(y_test_class, y_pred_class, digits=4))\n    report = classification_report(y_test_class, y_pred_class, output_dict=True, zero_division=0)\n\n    #confusion matrix\n    print(pd.crosstab(y_test_class, y_pred_class, colnames=[\'Predictions\']))\n\n    labels = np.unique(np.concatenate([y_train, y_val]))\n\n    accuracy = accuracy_score(y_test_class, y_pred_class)\n    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(\n        y_test_class, y_pred_class, average=\'macro\', zero_

In [ ]:
plot_training_history(history, "reports/figures/training_history/", model_name)